# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_decoding_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import raster_plot, raster_plot_3d

# import decoding
from neural_data_analysis.neural_analysis_tools.decoding_tools.event_decoding import decoding_utils, decoding_analysis, plot_decoding, cmp_decode, load_results
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_neural_viz import stop_event_raster
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import stop_time_warp

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)

# retrieve data

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0301"

pn, datasets, comparisons = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path)

globals().update(datasets)

captures_df, valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    distance_col="distance_from_ff_to_stop",
)

keys = ['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle', 'one_stop_vs_both_first_miss', 'will_switch_vs_retry_after_miss', 'switch_vs_retry_after_retry']

cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.05,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

# Single stops

## new_seg_info

In [ ]:
stops_with_stats['stop_time'] = stops_with_stats['stop_id_start_time']
stops_with_stats['prev_time'] = stops_with_stats['stop_id_end_time'].shift(1)
stops_with_stats['next_time'] = stops_with_stats['stop_id_start_time'].shift(-1)

new_seg_info = event_binning.pick_event_window(stops_with_stats,
                                                pre_s=0.2, post_s=1.0, min_pre_bins=1, min_post_bins=20, bin_dt=0.04)

if 'stop_id' not in pn.closest_stop_to_capture_df.columns:
    pn.closest_stop_to_capture_df = get_stops_utils.add_stop_id_to_closest_stop_to_capture_df(
        pn.closest_stop_to_capture_df,
        pn.monkey_information,
    )
    
if 'captured' not in new_seg_info.columns:
    pn.closest_stop_to_capture_df['captured'] = 1
    new_seg_info = new_seg_info.merge(pn.closest_stop_to_capture_df[['stop_id', 'captured']].drop_duplicates(), on='stop_id', how='left')
    new_seg_info['captured'] = new_seg_info['captured'].fillna(0)
    
new_seg_info['event_id'] = new_seg_info['stop_id']
new_seg_info['event_time'] = new_seg_info['stop_time']

# after using pick_event_window to select events (that have enough time apart from the event before and after),
# we make new start and end time
# # distribute data into new_segments and add rel_spike_time
new_seg_info = new_seg_info.sort_values('stop_id_duration', ascending=True).reset_index(drop=True)
new_seg_info['new_segment'] = np.arange(len(new_seg_info))
new_seg_info['new_seg_start_time'] = new_seg_info['stop_id_start_time'] - 1
new_seg_info['new_seg_end_time'] = new_seg_info['stop_id_end_time'] + 1
new_seg_info['next_stop_time'] = new_seg_info['next_time']


In [ ]:
aligned_spike_trains = raster_plot.prepare_aligned_spike_trains(new_seg_info, pn.spikes_df,
                                                                extra_columns_to_preserve=['event_time', 'stop_time', 'next_stop_time',
                                                                                     'stop_id_end_time'])

reference_time_col = 'event_time'
scale_anchor_col = 'event_time'
scale_factor_lower_col = 'stop_time'
scale_factor_upper_col = 'stop_id_end_time'

aligned_spike_trains, new_seg_info = raster_plot.add_relative_times([aligned_spike_trains, new_seg_info], reference_time_col='event_time')

aligned_spike_trains, new_seg_info = raster_plot.add_scaling_info([aligned_spike_trains, new_seg_info],
                                                    scale_anchor_col=scale_anchor_col,
                                                    scale_factor_lower_col=scale_factor_lower_col,
                                                    scale_factor_upper_col=scale_factor_upper_col)

In [ ]:
events_with_stats = stops_with_stats[['stop_id','stop_cluster_id','stop_id_start_time','stop_id_end_time']].copy()
events_with_stats.rename(columns={'stop_id':'event_id', 'stop_cluster_id':'event_cluster_id', 
                                  'stop_id_start_time':'event_id_start_time', 
                                  'stop_id_end_time':'event_id_end_time'}, inplace=True)

## what if only keep stop clusters w captured

In [ ]:
# cluster_id_w_captures = new_seg_info.loc[new_seg_info['captured'] == 1, 'stop_cluster_id'].unique()
# new_seg_info = new_seg_info[new_seg_info['stop_cluster_id'].isin(cluster_id_w_captures)].copy()

In [ ]:
# new_seg_info = new_seg_info[new_seg_info['captured'] == 1].copy()
# new_seg_info

## plot raster

### by cluster

In [ ]:

raster_plot.plot_rasters(aligned_spike_trains, 
                         new_seg_info=new_seg_info,
                         max_clusters_to_plot=2,
                         scale_spike_times=False,
                         #col_to_rearrange_segments='rel_stop_time',
                         xmin=-0.5,
                         xmax=3,
                         events_to_plot=[
                                        #'rel_event_time', 
                                        'rel_stop_time', 
                                        #'rel_prev_ff_caught_time',
                                        #'rel_new_seg_start_time', 
                                        'rel_stop_id_end_time',
                                        'rel_new_seg_end_time',
                                        'rel_next_stop_time',
                                        ])

### by segment

In [ ]:
raster_plot.plot_rasters_by_segment(aligned_spike_trains, 
                                    new_seg_info=new_seg_info,
                         scale_spike_times=False,
                         #col_to_rearrange_segments='rel_stop_time',
                         xmin=-0.5,
                         xmax=2,
                         segments_to_plot=np.arange(3, 5),
                         events_to_plot=[
                                        #'rel_event_time', 
                                        'rel_stop_time', 
                                        #'rel_prev_ff_caught_time',
                                        #'rel_new_seg_start_time', 
                                        'rel_stop_id_end_time',
                                        'rel_new_seg_end_time',
                                        'rel_next_stop_time',
                                        ])

# Retry series (Rsw)

## new_seg_info

In [ ]:
from pattern_discovery.learning.retry_analysis import prep_retry_data
from decision_making_analysis.data_compilation import miss_events_class

In [ ]:
cgt = miss_events_class.MissEventsClass(
    raw_data_folder_path=raw_data_folder_path,)
cgt.get_monkey_data(already_retrieved_ok=True, include_ff_dataframe=False, include_rsw_data=True,
                    include_rcap_data=True)

In [ ]:
added stop_time_warp.py as well as time warped raster plots to stop_event_rasters.

In [ ]:
retry_new_seg_info, stop_time_cols, stop_end_cols = stop_event_raster.build_retry_new_seg_info(cgt.rsw_events_df, pre_s=0.5, post_s=0.5)

In [ ]:
reference_time_col = 'stop_1_time'
scale_anchor_col = 'stop_1_time'
scale_factor_lower_col = 'stop_1_time'
scale_factor_upper_col = 'last_stop_time'

retry_aligned_spike_trains = raster_plot.prepare_aligned_spike_trains(retry_new_seg_info, pn.spikes_df,
                                                                extra_columns_to_preserve=['event_time', 'stop_1_id_duration',
                                                                                           reference_time_col, scale_anchor_col, 
                                                                                           scale_factor_lower_col, scale_factor_upper_col])



retry_aligned_spike_trains, retry_new_seg_info = raster_plot.add_relative_times([retry_aligned_spike_trains, retry_new_seg_info], reference_time_col='event_time')

retry_aligned_spike_trains, retry_new_seg_info = raster_plot.add_scaling_info([retry_aligned_spike_trains, retry_new_seg_info],
                                                    scale_anchor_col=scale_anchor_col,
                                                    scale_factor_lower_col=scale_factor_lower_col,
                                                    scale_factor_upper_col=scale_factor_upper_col)

# Warped

## Get data

In [ ]:
warper = stop_time_warp.StopAlignedSpikeWarp(retry_new_seg_info, K=3)

warped_spikes = warper.warp_spikes(retry_aligned_spike_trains)
warped_spikes['spike_time'] += warped_spikes['new_seg_start_time']

aligned_spike_trains_warped, new_seg_info_warped = warper.propagate_template_stops(
    spike_df=warped_spikes,
    seg_info_df=retry_new_seg_info,
    max_num_stops=3
)


aligned_spike_trains_warped['event_time'] = aligned_spike_trains_warped['stop_1_time']
new_seg_info_warped['event_time'] = new_seg_info_warped['stop_1_time']

if 'num_stops' not in aligned_spike_trains_warped.columns:
    aligned_spike_trains_warped = aligned_spike_trains_warped.merge(new_seg_info_warped[['new_segment', 'num_stops']], on='new_segment', how='left')


aligned_spike_trains_warped, new_seg_info_warped = raster_plot.add_relative_times([aligned_spike_trains_warped, new_seg_info_warped], reference_time_col='event_time')

aligned_spike_trains_warped, new_seg_info_warped = raster_plot.add_scaling_info([aligned_spike_trains_warped, new_seg_info_warped],
                                                    scale_anchor_col=scale_anchor_col,
                                                    scale_factor_lower_col=scale_factor_lower_col,
                                                    scale_factor_upper_col=scale_factor_upper_col)

## Squashed

### by segment

In [ ]:
raster_plot.plot_rasters_by_segment(aligned_spike_trains_warped, 
                                    new_seg_info=new_seg_info_warped,
                                    scale_spike_times=False,
                                    # col_to_rearrange_segments='num_stops',
                                    xmin=-0.5,
                                    xmax=2,
                                    segments_to_plot=np.arange(10, 40),
                                    shade_start_cols=stop_time_cols,
                                    shade_end_cols=stop_end_cols,
                                    events_to_plot=[],
                                    combine_subplots=True,
                                    show_cluster_tick_labels=False
                                    )

In [ ]:
stop!

### by cluster

Btw...you don't have to plot all segments at once here

In [ ]:
raster_plot.plot_rasters(aligned_spike_trains_warped, 
                         new_seg_info=new_seg_info_warped,
                         col_to_rearrange_segments='num_stops',
                         max_clusters_to_plot=20,
                         scale_spike_times=False,
                         xmin=-0.5,
                         xmax=3,
                         events_to_plot=[],
                         #events_to_plot=['rel_' + c for c in stop_time_cols],
                         shade_start_cols=stop_time_cols,
                         shade_end_cols=stop_end_cols,
                         combine_subplots=True
                         )

In [ ]:
stop!

## Just subplots

### by cluster

In [ ]:
raster_plot.plot_rasters(aligned_spike_trains_warped, 
                         new_seg_info=new_seg_info_warped,
                         col_to_rearrange_segments='num_stops',
                         max_clusters_to_plot=30,
                         scale_spike_times=False,
                         xmin=-0.5,
                         xmax=3,
                         events_to_plot=[],
                         #events_to_plot=['rel_' + c for c in stop_time_cols],
                         shade_start_cols=stop_time_cols,
                         shade_end_cols=stop_end_cols)

### by segment

In [ ]:
raster_plot.plot_rasters_by_segment(aligned_spike_trains_warped, 
                         new_seg_info=new_seg_info_warped,
                         scale_spike_times=False,
                         #col_to_rearrange_segments='rel_stop_time',
                         xmin=-0.5,
                         xmax=2,
                         segments_to_plot=np.arange(10, 60),
                         shade_start_cols=stop_time_cols,
                         shade_end_cols=stop_end_cols,
                         events_to_plot=[],
                         show_cluster_tick_labels=False
                         )

In [ ]:
stop!

# Original

## Squashed

### by segment

In [ ]:
raster_plot.plot_rasters_by_segment(retry_aligned_spike_trains, 
                                    new_seg_info=retry_new_seg_info,
                                    scale_spike_times=False,
                                    col_to_rearrange_segments='num_stops',
                                    xmin=-0.5,
                                    xmax=2,
                                    segments_to_plot=np.arange(10, 40),
                                    shade_start_cols=stop_time_cols,
                                    shade_end_cols=stop_end_cols,
                                    events_to_plot=[],
                                    combine_subplots=True,
                                    show_cluster_tick_labels=False
                                    )

In [ ]:
stop!

### by cluster

Btw...you don't have to plot all segments at once here

In [ ]:
raster_plot.plot_rasters(retry_aligned_spike_trains, 
                         new_seg_info=retry_new_seg_info,
                         col_to_rearrange_segments='num_stops',
                         max_clusters_to_plot=20,
                         scale_spike_times=False,
                         xmin=-0.5,
                         xmax=3,
                         events_to_plot=[],
                         #events_to_plot=['rel_' + c for c in stop_time_cols],
                         shade_start_cols=stop_time_cols,
                         shade_end_cols=stop_end_cols,
                         combine_subplots=True
                         )

In [ ]:
stop!

## Just subplots

### by cluster

In [ ]:
raster_plot.plot_rasters(retry_aligned_spike_trains, 
                         new_seg_info=retry_new_seg_info,
                         col_to_rearrange_segments='num_stops',
                         max_clusters_to_plot=30,
                         scale_spike_times=False,
                         xmin=-0.5,
                         xmax=3,
                         events_to_plot=[],
                         #events_to_plot=['rel_' + c for c in stop_time_cols],
                         shade_start_cols=stop_time_cols,
                         shade_end_cols=stop_end_cols)

### by segment

In [ ]:
raster_plot.plot_rasters_by_segment(retry_aligned_spike_trains, 
                                    new_seg_info=retry_new_seg_info,
                         scale_spike_times=False,
                         #col_to_rearrange_segments='rel_stop_time',
                         xmin=-0.5,
                         xmax=2,
                         segments_to_plot=np.arange(10, 60),
                         shade_start_cols=stop_time_cols,
                         shade_end_cols=stop_end_cols,
                         events_to_plot=[],
                         show_cluster_tick_labels=False
                         )

# 3D

In [ ]:
stop!

In [ ]:
raster_plot_3d.plot_3d_raster_plotly(
    retry_aligned_spike_trains,
    retry_new_seg_info,
    event_start_cols=stop_time_cols,
    event_end_cols=stop_end_cols,
    max_segments=10,
    max_neurons=15
)


# Appendix

## Dist of stop durations

In [ ]:
data = new_seg_info['stop_id_duration']
for perc in [0, 25, 50, 90, 95, 98, 99, 99.5, 99.9, 100]:
    print(f'{perc}th percentile: {np.percentile(data, perc)}')

sns.histplot(data)
plt.xlim(0, 5)
plt.show()